In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from tensorflow.keras.optimizers import SGD
import numpy as np
from numpy import save
from numpy import asarray
from numpy import load
import random
from tensorflow.keras.callbacks import LearningRateScheduler
from skimage import io, color, exposure, transform
from sklearn.model_selection import train_test_split
import os
import glob
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
img_rows, img_cols, channels = 48, 48, 3
num_classes = 43
def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]

    # rescale to standard size
    img = transform.resize(img, (img_rows, img_cols))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img

In [3]:
def get_class(img_path):
    return int(img_path.split('/')[-2])

root_dir = 'GTSRB_dataset/Final_Training/Images/'
imgs = []
labels = []
#load train dataset
all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)

x_train = np.array(imgs, dtype='float32')
# Make one hot targets
y_train = np.eye(num_classes, dtype='uint8')[labels]


In [4]:
test = pd.read_csv('GT-final_test.csv', sep=';')

# Load test dataset
x_test = []
y_test = []
i = 0
for file_name, class_id in zip(list(test['Filename']), list(test['ClassId'])):
    img_path = os.path.join('GTSRB_dataset/Final_Test/Images/', file_name)
    x_test.append(preprocess_img(io.imread(img_path)))
    y_test.append(class_id)

x_test = np.array(x_test)
y_test = np.array(y_test)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train = x_train.reshape((x_train.shape[0], img_rows, img_cols, channels))
x_test = x_test.reshape((x_test.shape[0], img_rows, img_cols, channels))

lr = 0.001
batch_size = 32
epochs = 30
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_rows, img_cols, channels), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

model = create_model()

In [6]:
model.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test))

Train on 39209 samples, validate on 12630 samples
Epoch 1/30
39209/39209 [==============================] - 101s 3ms/sample - loss: 3.5314 - accuracy: 0.0619 - val_loss: 3.4434 - val_accuracy: 0.0914
Epoch 2/30
39209/39209 [==============================] - 100s 3ms/sample - loss: 3.2183 - accuracy: 0.1384 - val_loss: 2.7479 - val_accuracy: 0.2271
Epoch 3/30
39209/39209 [==============================] - 100s 3ms/sample - loss: 2.5981 - accuracy: 0.2521 - val_loss: 2.2280 - val_accuracy: 0.3302
Epoch 4/30
39209/39209 [==============================] - 100s 3ms/sample - loss: 2.1217 - accuracy: 0.3533 - val_loss: 1.8999 - val_accuracy: 0.4008
Epoch 5/30
39209/39209 [==============================] - 100s 3ms/sample - loss: 1.7718 - accuracy: 0.4350 - val_loss: 1.6083 - val_accuracy: 0.5004
Epoch 6/30
39209/39209 [==============================] - 98s 3ms/sample - loss: 1.4721 - accuracy: 0.5228 - val_loss: 1.3388 - val_accuracy: 0.5788
Epoch 7/30
39209/39209 [===========================

In [7]:
m1=tf.keras.models.clone_model(model)
lr = 0.001
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
m1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
m1.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test))
m1.save('final_model_clean.h5')

Train on 39209 samples, validate on 12630 samples
Epoch 1/30
39209/39209 [==============================] - 72s 2ms/sample - loss: 3.5242 - accuracy: 0.0601 - val_loss: 3.4565 - val_accuracy: 0.1010
Epoch 2/30
39209/39209 [==============================] - 72s 2ms/sample - loss: 3.2312 - accuracy: 0.1359 - val_loss: 2.7677 - val_accuracy: 0.2282
Epoch 3/30
39209/39209 [==============================] - 78s 2ms/sample - loss: 2.6068 - accuracy: 0.2521 - val_loss: 2.2761 - val_accuracy: 0.3268
Epoch 4/30
39209/39209 [==============================] - 77s 2ms/sample - loss: 2.1391 - accuracy: 0.3482 - val_loss: 1.8744 - val_accuracy: 0.4224
Epoch 5/30
39209/39209 [==============================] - 73s 2ms/sample - loss: 1.7611 - accuracy: 0.4398 - val_loss: 1.5671 - val_accuracy: 0.5019
Epoch 6/30
39209/39209 [==============================] - 71s 2ms/sample - loss: 1.4192 - accuracy: 0.5438 - val_loss: 1.2602 - val_accuracy: 0.6028
Epoch 7/30
39209/39209 [==============================] 

In [8]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Base accuracy on regular images:", acc)
print("Base loss on regular images:", loss)

Base accuracy on regular images: 0.9364212
Base loss on regular images: 0.3147545412508734


In [9]:
def adversarial_pattern(image, label):
    image = tf.cast(image, tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label, prediction)
    
    gradient = tape.gradient(loss, image)
    
    signed_grad = tf.sign(gradient)
    
    return signed_grad

In [10]:
def generate_adversarials(batch_size):
    while True:
        x = []
        y = []
        for batch in range(batch_size):
            #N = random.randint(0, 100)

            label = y_test[batch]
            image = x_test[batch]
            
            perturbations = adversarial_pattern(image.reshape((1, img_rows, img_cols, channels)), label).numpy()
            
            
            epsilon = 0.3
            adversarial = image + perturbations * epsilon
            
            x.append(adversarial)
            y.append(y_test[batch])
        
        
        x = np.asarray(x).reshape((batch_size, img_rows, img_cols, channels))
        y = np.asarray(y)
        
        yield x, y

In [11]:
x_adversarial_test, y_adversarial_test = next(generate_adversarials(12630))

In [12]:
loss1, acc1 = model.evaluate(x_test, y_test, verbose=0)
print("Base accuracy on regular images:", acc1)
print("Base loss on regular images:", loss1)
loss2, acc2 = model.evaluate(x_adversarial_test, y_adversarial_test, verbose=0)
print("Base accuracy on adversarial images:",acc2)
print('Base loss on adversarial images:', loss2)

Base accuracy on regular images: 0.9364212
Base loss on regular images: 0.3147545412508734
Base accuracy on adversarial images: 0.30451307
Base loss on adversarial images: 14.670813890005611


In [13]:
save('adversarial_test_images',x_adversarial_test)
save('adversarial_test_labels',y_adversarial_test)
save('clean_test_images',x_test)
save('clean_test_labels',y_test)
model.save('final_model_adv.h5')